In [40]:
import pandas as pd
import numpy as np

In [5]:
train = pd.read_csv('C:/users/micha/Desktop/Bemidji Meetup/titanic/train.csv')
test = pd.read_csv('C:/users/micha/Desktop/Bemidji Meetup/titanic/test.csv')

In [7]:
print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


In [23]:
# we can create a submission file for later with the Passenger ID
sub = pd.DataFrame(data= test.PassengerId)

In [24]:
sub.head()

,PassengerId
0,892
1,893
2,894
3,895
4,896


In [29]:
# concat both dataframes for easier data manipulation
combined = pd.concat([train,test])

In [30]:
combined.head(3)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282


In [31]:
# starting with age
combined.Age.describe()

count    1046.000000
mean       29.881138
std        14.413493
min         0.170000
25%        21.000000
50%        28.000000
75%        39.000000
max        80.000000
Name: Age, dtype: float64

In [32]:
# there doesnt appear to be any major outliers on age but we do have some missing values
combined[combined.Age.isnull()]

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
5,NaN,NaN,Q,8.4583,"Moran, Mr. James",0,6,3,male,0,0.0,330877
17,NaN,NaN,S,13.0000,"Williams, Mr. Charles Eugene",0,18,2,male,0,1.0,244373
19,NaN,NaN,C,7.2250,"Masselmani, Mrs. Fatima",0,20,3,female,0,1.0,2649
26,NaN,NaN,C,7.2250,"Emir, Mr. Farred Chehab",0,27,3,male,0,0.0,2631
28,NaN,NaN,Q,7.8792,"O'Dwyer, Miss. Ellen ""Nellie""",0,29,3,female,0,1.0,330959
29,NaN,NaN,S,7.8958,"Todoroff, Mr. Lalio",0,30,3,male,0,0.0,349216
31,NaN,B78,C,146.5208,"Spencer, Mrs. William Augustus (Marie Eugenie)",0,32,1,female,1,1.0,PC 17569
32,NaN,NaN,Q,7.7500,"Glynn, Miss. Mary Agatha",0,33,3,female,0,1.0,335677
36,NaN,NaN,C,7.2292,"Mamee, Mr. Hanna",0,37,3,male,0,1.0,2677
42,NaN,NaN,C,7.8958,"Kraeff, Mr. Theodor",0,43,3,male,0,0.0,349253


In [33]:
# we have 263 missing values.  If you recall, last time we imputed the missing ages with a mean - 29 years old
# we are going to take a different approach this time around becasue age is likely very important
# I think a good indicator of age may be title, so lets extract titles first
combined['Title'] = combined['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
# and we can drop Name and while we are at, passenger id
combined.drop(['Name', 'PassengerId', 'Ticket'], axis=1, inplace=True)

In [34]:
combined.head(3)

,Age,Cabin,Embarked,Fare,Parch,Pclass,Sex,SibSp,Survived,Title
0,22.0,NaN,S,7.2500,0,3,male,1,0.0,Mr
1,38.0,C85,C,71.2833,0,1,female,1,1.0,Mrs
2,26.0,NaN,S,7.9250,0,3,female,0,1.0,Miss


In [41]:
combined.Title.value_counts(dropna=False)

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Ms                2
Major             2
Sir               1
Mme               1
Capt              1
Jonkheer          1
Don               1
the Countess      1
Lady              1
Dona              1
Name: Title, dtype: int64

In [57]:
MrAge = combined.loc[combined.Title=='Mr', 'Age'].mean()
MissAge = combined.loc[combined.Title=='Miss', 'Age'].mean()
MrsAge = combined.loc[combined.Title=='Mrs', 'Age'].mean()
MasterAge = combined.loc[combined.Title=='Master', 'Age'].mean()
RevAge = combined.loc[combined.Title=='Rev', 'Age'].mean()
DrAge = combined.loc[combined.Title=='Dr', 'Age'].mean()
ColAge = combined.loc[combined.Title=='Col', 'Age'].mean()
MlleAge = combined.loc[combined.Title=='Mlle', 'Age'].mean()
MsAge = combined.loc[combined.Title=='Ms', 'Age'].mean()
MajorAge = combined.loc[combined.Title=='Major', 'Age'].mean()
SirAge = combined.loc[combined.Title=='Sir', 'Age'].mean()
MmeAge = combined.loc[combined.Title=='Mme', 'Age'].mean()
CaptAge = combined.loc[combined.Title=='Capt', 'Age'].mean()
JonkheerAge = combined.loc[combined.Title=='Jonkheer', 'Age'].mean()
DonAge = combined.loc[combined.Title=='Don', 'Age'].mean()
CountessAge = combined.loc[combined.Title=='the Countess', 'Age'].mean()
LadyAge = combined.loc[combined.Title=='Lady', 'Age'].mean()
DonaAge = combined.loc[combined.Title=='Dona', 'Age'].mean()